Copyright (c) Microsoft Corporation. All rights reserved.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/regression-part2-automated-ml.png)

# HOL02: Azure Machine Learning serice

This lab guides you through Azure Machine Learning service - creation & setup, building experiment and train model (also use automated machine learning technique).

In this use case you build a regression model to predict NYC taxi fare prices. 
This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

In this lab you learn the following tasks:

* Create Azure Machine Learning Workspace
* Download, transform, and clean data using Azure Open Datasets
* Train an automated machine learning regression model
* Calculate model accuracy

If you don’t have an Azure subscription, create a free account before you begin. Try the [free or paid version](https://aka.ms/AMLFree) of Azure Machine Learning service today.

## Prerequisites

* Existing AML Workspace - step can be found on GitHub repo for this lab []()


## Download and prepare data

Import the necessary packages. The Open Datasets package contains a class representing each data source (`NycTlcGreen` for example) to easily filter date parameters before downloading.

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid `MemoryError` with large datasets. To download taxi data, iteratively fetch one month at a time, and before appending it to `green_taxi_df` randomly sample 2,000 records from each month to avoid bloating the dataframe. Then preview the data.

In [2]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))
    
green_taxi_df.head(10)

ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=1/part-00175-tid-3144095708418575143-7b1fb9db-f589-44c3-b42e-832eaf9f427c-21996.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=7701.77 [ms]
ActivityCompleted: Activity=to_pandas_dataframe, HowEnded=Success, Duration=7721.91 [ms]
ActivityStarted, to_pandas_dataframe
ActivityStarted, to_pandas_dataframe_in_worker
Target paths: ['/puYear=2015/puMonth=2/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading green/puYear=2015/puMonth=2/part-00007-tid-3144095708418575143-7b1fb9db-f589-44c3-b42e-832eaf9f427c-21828.c000.snappy.parquet under container nyctlc
Done.
ActivityCompleted: Activity=to_pandas_dataframe_in_worker, HowEnded=Success, Duration=7652.18 [ms]
A

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
237225,2,2015-01-13 19:10:25,2015-01-13 19:14:48,1,0.78,None,None,-73.913727,40.755795,-73.914711,...,1,5.0,1.0,0.5,0.3,1.50,0.0,NaN,8.30,1.0
987639,1,2015-01-11 00:33:13,2015-01-11 00:55:58,1,6.90,None,None,-73.954788,40.716537,-73.970329,...,1,23.5,0.5,0.5,0.3,3.00,0.0,NaN,27.80,1.0
202187,2,2015-01-12 22:18:11,2015-01-12 22:45:41,1,7.43,None,None,-73.976387,40.682281,-73.993675,...,2,26.5,0.5,0.5,0.3,0.00,0.0,NaN,27.80,1.0
690834,2,2015-01-02 13:49:16,2015-01-02 13:54:03,1,1.23,None,None,-73.962196,40.805347,-73.950836,...,1,6.0,0.0,0.5,0.3,1.20,0.0,NaN,8.00,1.0
408921,2,2015-01-22 15:30:43,2015-01-22 15:37:50,1,1.61,None,None,-73.951164,40.791225,-73.947899,...,2,7.5,0.0,0.5,0.3,0.00,0.0,NaN,8.30,1.0
508958,2,2015-01-24 13:07:56,2015-01-24 13:13:17,1,0.70,None,None,-73.974525,40.671333,-73.970139,...,1,5.5,0.0,0.5,0.3,1.26,0.0,NaN,7.56,1.0
1387997,1,2015-01-28 00:57:33,2015-01-28 01:15:43,1,4.20,None,None,-73.844284,40.721405,-73.831825,...,1,16.5,0.5,0.5,0.3,3.55,0.0,NaN,21.35,1.0
756008,2,2015-01-03 19:11:11,2015-01-03 19:16:22,1,0.79,None,None,-73.967033,40.683578,-73.968079,...,2,5.5,0.0,0.5,0.3,0.00,0.0,NaN,6.30,1.0
1127641,2,2015-01-20 14:53:05,2015-01-20 15:01:07,1,1.14,None,None,-73.977936,40.684486,-73.972984,...,2,7.0,0.0,0.5,0.3,0.00,0.0,NaN,7.80,1.0
599474,2,2015-01-29 21:07:20,2015-01-29 21:27:28,3,6.01,None,None,-73.950157,40.809776,-73.983223,...,1,21.0,0.5,0.5,0.3,4.46,0.0,NaN,26.76,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day, and will allow the model to factor in time-based seasonality. 

Use the `apply()` function on the dataframe to iteratively apply the `build_time_features()` function to each row in the taxi data.

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
237225,2,2015-01-13 19:10:25,2015-01-13 19:14:48,1,0.78,None,None,-73.913727,40.755795,-73.914711,...,0.3,1.50,0.0,NaN,8.30,1.0,1,13,1,19
987639,1,2015-01-11 00:33:13,2015-01-11 00:55:58,1,6.90,None,None,-73.954788,40.716537,-73.970329,...,0.3,3.00,0.0,NaN,27.80,1.0,1,11,6,0
202187,2,2015-01-12 22:18:11,2015-01-12 22:45:41,1,7.43,None,None,-73.976387,40.682281,-73.993675,...,0.3,0.00,0.0,NaN,27.80,1.0,1,12,0,22
690834,2,2015-01-02 13:49:16,2015-01-02 13:54:03,1,1.23,None,None,-73.962196,40.805347,-73.950836,...,0.3,1.20,0.0,NaN,8.00,1.0,1,2,4,13
408921,2,2015-01-22 15:30:43,2015-01-22 15:37:50,1,1.61,None,None,-73.951164,40.791225,-73.947899,...,0.3,0.00,0.0,NaN,8.30,1.0,1,22,3,15
508958,2,2015-01-24 13:07:56,2015-01-24 13:13:17,1,0.70,None,None,-73.974525,40.671333,-73.970139,...,0.3,1.26,0.0,NaN,7.56,1.0,1,24,5,13
1387997,1,2015-01-28 00:57:33,2015-01-28 01:15:43,1,4.20,None,None,-73.844284,40.721405,-73.831825,...,0.3,3.55,0.0,NaN,21.35,1.0,1,28,2,0
756008,2,2015-01-03 19:11:11,2015-01-03 19:16:22,1,0.79,None,None,-73.967033,40.683578,-73.968079,...,0.3,0.00,0.0,NaN,6.30,1.0,1,3,5,19
1127641,2,2015-01-20 14:53:05,2015-01-20 15:01:07,1,1.14,None,None,-73.977936,40.684486,-73.972984,...,0.3,0.00,0.0,NaN,7.80,1.0,1,20,1,14
599474,2,2015-01-29 21:07:20,2015-01-29 21:27:28,3,6.01,None,None,-73.950157,40.809776,-73.983223,...,0.3,4.46,0.0,NaN,26.76,1.0,1,29,3,21


Remove some of the columns that you won't need for training or additional feature building.

In [4]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)
    
green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
237225,2,1,0.78,-73.913727,40.755795,-73.914711,40.763794,8.3,1,13,1,19
987639,1,1,6.90,-73.954788,40.716537,-73.970329,40.762360,27.8,1,11,6,0
202187,2,1,7.43,-73.976387,40.682281,-73.993675,40.745811,27.8,1,12,0,22
690834,2,1,1.23,-73.962196,40.805347,-73.950836,40.820660,8.0,1,2,4,13
408921,2,1,1.61,-73.951164,40.791225,-73.947899,40.809391,8.3,1,22,3,15


### Cleanse data 

Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [5]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.00000,24000.000000
mean,1.782917,1.368208,2.880052,-73.830472,40.691962,-73.813779,40.681474,14.718021,6.500000,15.164375,3.28050,13.540542
std,0.412268,1.030942,2.881211,2.781231,1.533798,2.978444,1.642351,11.159739,3.452124,8.476429,1.95814,6.794105
min,1.000000,0.000000,0.000000,-74.215767,0.000000,-74.409607,0.000000,-52.800000,1.000000,1.000000,0.00000,0.000000
25%,2.000000,1.000000,1.090000,-73.959084,40.698847,-73.967356,40.699240,8.000000,3.750000,8.000000,2.00000,9.000000
50%,2.000000,1.000000,1.920000,-73.945023,40.746906,-73.944290,40.748310,11.300000,6.500000,15.000000,3.00000,15.000000
75%,2.000000,1.000000,3.660000,-73.917128,40.803784,-73.909035,40.793303,17.800000,9.250000,22.000000,5.00000,19.000000
max,2.000000,8.000000,55.160000,0.000000,40.922737,0.000000,41.034153,399.800000,12.000000,30.000000,6.00000,23.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the bounds of the Manhattan area. This will filter out longer taxi trips or trips that are outliers in respect to their relationship with other features. 

Additionally filter the `tripDistance` field to be greater than zero but less than 31 miles (the haversine distance between the two lat/long pairs). This eliminates long outlier trips that have inconsistent trip cost.

Lastly, the `totalAmount` field has negative values for the taxi fares, which don't make sense in the context of our model, and the `passengerCount` field has bad data with the minimum values being zero.

Filter out these anomalies using query functions, and then remove the last few columns unnecessary for training.

In [ ]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

Call `describe()` again on the data to ensure cleansing worked as expected. You now have a prepared and cleansed set of taxi, holiday, and weather data to use for machine learning model training.

In [ ]:
final_df.describe()

## Configure workspace


Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

## Split the data into train and test sets

Split the data into training and test sets by using the `train_test_split` function in the `scikit-learn` library. This function segregates the data into the x (**features**) data set for model training and the y (**values to predict**) data set for testing. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random generator, so that your train-test splits are deterministic.

In [ ]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("totalAmount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

The purpose of this step is to have data points to test the finished model that haven't been used to train the model, in order to measure true accuracy. 

In other words, a well-trained model should be able to accurately make predictions from data it hasn't already seen. You now have data prepared for auto-training a machine learning model.

## Automatically train a model

To automatically train a model, take the following steps:
1. Define settings for the experiment run. Attach your training data to the configuration, and modify settings that control the training process.
1. Submit the experiment for model tuning. After submitting the experiment, the process iterates through different machine learning algorithms and hyperparameter settings, adhering to your defined constraints. It chooses the best-fit model by optimizing an accuracy metric.

### Define training settings

Define the experiment parameter and model settings for training. View the full list of [settings](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train). Submitting the experiment with these default settings will take approximately 5-10 min, but if you want a shorter run time, reduce the `iterations` parameter.


|Property| Value in this tutorial |Description|
|----|----|---|
|**iteration_timeout_minutes**|2|Time limit in minutes for each iteration. Reduce this value to decrease total runtime.|
|**iterations**|20|Number of iterations. In each iteration, a new machine learning model is trained with your data. This is the primary value that affects total run time.|
|**primary_metric**| spearman_correlation | Metric that you want to optimize. The best-fit model will be chosen based on this metric.|
|**preprocess**| True | By using **True**, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)|
|**verbosity**| logging.INFO | Controls the level of logging.|
|**n_cross_validations**|5|Number of cross-validation splits to perform when validation data is not specified.|

In [ ]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "iterations": 20,
    "primary_metric": 'spearman_correlation',
    "preprocess": True,
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

Use your defined training settings as a `**kwargs` parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

Automated machine learning pre-processing steps (feature normalization, handling missing data, converting text to numeric, etc.) become part of the underlying model. When using the model for predictions, the same pre-processing steps applied during training are applied to your input data automatically.

### Train the automatic regression model

Create an experiment object in your workspace. An experiment acts as a container for your individual runs. Pass the defined `automl_config` object to the experiment, and set the output to `True` to view progress during the run. 

After starting the experiment, the output shown updates live as the experiment runs. For each iteration, you see the model type, the run duration, and the training accuracy. The field `BEST` tracks the best running training score based on your metric type.

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

## Explore the results

Explore the results of automatic training with a [Jupyter widget](https://docs.microsoft.com/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py). The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function `predict` uses the best model and predicts the values of y, **trip cost**, from the `x_test` data set. Print the first 10 predicted cost values from `y_predict`.

In [ ]:
y_predict = fitted_model.predict(x_test.values)
print(y_predict[:10])

Calculate the `root mean squared error` of the results. Convert the `y_test` dataframe to a list to compare to the predicted values. The function `mean_squared_error` takes two arrays of values and calculates the average squared error between them. Taking the square root of the result gives an error in the same units as the y variable, **cost**. It indicates roughly how far the taxi fare predictions are from the actual fares.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

Run the following code to calculate mean absolute percent error (MAPE) by using the full `y_actual` and `y_predict` data sets. This metric calculates an absolute difference between each predicted and actual value and sums all the differences. Then it expresses that sum as a percent of the total of the actual values.

In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error. 

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.

## Clean up resources

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Notebook VMs**.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this automated machine learning tutorial, you did the following tasks:

> * Configured a workspace and prepared data for an experiment.
> * Trained by using an automated regression model locally with custom parameters.
> * Explored and reviewed training results.

[Deploy your model](https://docs.microsoft.com/azure/machine-learning/service/tutorial-deploy-models-with-aml) with Azure Machine Learning service.